In [1]:
import pystac_client
from pystac_client.stac_api_io import StacApiIO

In [2]:
url = "https://api.stac.ceda.ac.uk/"

In [3]:
client = pystac_client.Client.open(url)

In [4]:
collection_name = "sentinel2_ard"

In [5]:
sentinel2_ard = client.get_collection(collection_name)

In [67]:
def doSearch(collection_name, start_date, end_date, bounding_box):
    item_search = client.search(
    collections=[collection_name],
    bbox=bounding_box,
    query=[f'start_datetime>={start_date}',
        f'end_datetime<={end_date}',
          ],
    max_items=10
    )
    return item_search

## Scotland
#bounding_box = [-3.003, 60.369, -0.945, 61.31]
## Entire UK
#bounding_box = [0, 50.28, 5, 59.94]
## Northern Ireland
bounding_box = [-9.0,53.5,-5.1,55.7]
search_result = doSearch("sentinel2_ard", "1900-01-01", "2023-11-28", bounding_box)

search_url = search_result.url_with_parameters()

print(search_url)

items = list(search_result.items())

items

https://api.stac.ceda.ac.uk/search?bbox=-9.0%2C53.5%2C-5.1%2C55.7&collections=sentinel2_ard&query=%7B%22start_datetime%22%3A%7B%22gte%22%3A%221900-01-01%22%7D%2C%22end_datetime%22%3A%7B%22lte%22%3A%222023-11-28%22%7D%7D


[<Item id=neodc.sentinel_ard.data.sentinel_2.2023.11.01.S2B_20231101_latn554lonw0081_T29UNB_ORB123_20231101123121_utm29n_TM65>,
 <Item id=neodc.sentinel_ard.data.sentinel_2.2023.11.01.S2B_20231101_latn554lonw0066_T29UPB_ORB123_20231101123121_utm29n_osgb>,
 <Item id=neodc.sentinel_ard.data.sentinel_2.2023.11.01.S2B_20231101_latn554lonw0066_T29UPB_ORB123_20231101123121_utm29n_TM65>,
 <Item id=neodc.sentinel_ard.data.sentinel_2.2023.11.01.S2B_20231101_latn554lonw0053_T30UUG_ORB123_20231101123121_utm30n_osgb>,
 <Item id=neodc.sentinel_ard.data.sentinel_2.2023.11.01.S2B_20231101_latn554lonw0053_T30UUG_ORB123_20231101123121_utm30n_TM65>,
 <Item id=neodc.sentinel_ard.data.sentinel_2.2023.11.01.S2B_20231101_latn546lonw0082_T29UNA_ORB123_20231101123121_utm29n_TM65>,
 <Item id=neodc.sentinel_ard.data.sentinel_2.2023.11.01.S2B_20231101_latn545lonw0066_T29UPA_ORB123_20231101123121_utm29n_TM65>,
 <Item id=neodc.sentinel_ard.data.sentinel_2.2023.11.01.S2B_20231101_latn545lonw0052_T30UUF_ORB123_20231

In [70]:
import json
import requests


response = requests.get(search_url)

json_out = response.json()
#print(json_out)
# each feature is a single bounding box here, and contains a variety of bands
features = json_out['features']

#print(features)
## each feature defines a region of data, then contains different spectral images over that region
for feature in features:
    instance_id = feature['properties']['instance_id']
    print(feature['assets'])
    asset_urls = [v['href'] for k,v in feature['assets'].items()]
    #print(len(asset_urls))

asset_items = json_out['features'][0]['assets']

#print(json_out['features'][0]['properties']['instance_id'])

#asset_urls = [v['href'] for k,v in asset_items.items()]

#asset_urls


{'S2B_20231101_latn554lonw0081_T29UNB_ORB123_20231101123121_utm29n_TM65_vmsk_sharp_rad_srefdem_stdsref_thumbnail.jpg': {'href': 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/01/S2B_20231101_latn554lonw0081_T29UNB_ORB123_20231101123121_utm29n_TM65_vmsk_sharp_rad_srefdem_stdsref_thumbnail.jpg', 'size': 72543, 'role': ['data'], 'location': 'on_disk'}, 'S2B_20231101_latn554lonw0081_T29UNB_ORB123_20231101123121_utm29n_TM65_vmsk_sharp_rad_srefdem_stdsref_meta.xml': {'href': 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/01/S2B_20231101_latn554lonw0081_T29UNB_ORB123_20231101123121_utm29n_TM65_vmsk_sharp_rad_srefdem_stdsref_meta.xml', 'size': 18546, 'role': ['data'], 'location': 'on_disk'}, 'S2B_20231101_latn554lonw0081_T29UNB_ORB123_20231101123121_utm29n_TM65_clouds_prob.tif': {'href': 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/01/S2B_20231101_latn554lonw0081_T29UNB_ORB123_20231101123121_utm29n_TM65_clouds_prob.tif', 'size': 644

In [51]:
import xarray as xr
#import netcdf4
import requests
import rioxarray

rds = rioxarray.open_rasterio(asset_urls[0], overview_level=3)
rds

<xarray.DataArray (band: 1, y: 696, x: 696)>
[484416 values with dtype=uint8]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 3.435e+05 3.436e+05 ... 4.546e+05 4.547e+05
  * y            (y) float64 1.173e+06 1.173e+06 ... 1.062e+06 1.062e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     0
    scale_factor:   1.0
    add_offset:     0.0

In [15]:
asset_items = items[0].assets
asset_items

asset_urls = [v.href for k,v in asset_items.items()]

asset_urls



['https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/01/S2B_20231101_latn608lonw0020_T30VWN_ORB123_20231101123121_utm30n_osgb_clouds_prob.tif',
 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/01/S2B_20231101_latn608lonw0020_T30VWN_ORB123_20231101123121_utm30n_osgb_sat.tif',
 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/01/S2B_20231101_latn608lonw0020_T30VWN_ORB123_20231101123121_utm30n_osgb_clouds.tif',
 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/01/S2B_20231101_latn608lonw0020_T30VWN_ORB123_20231101123121_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref_meta.xml',
 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/01/S2B_20231101_latn608lonw0020_T30VWN_ORB123_20231101123121_utm30n_osgb_valid.tif',
 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/01/S2B_20231101_latn608lonw0020_T30VWN_ORB123_20231101123121_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif',
 'https://dap.ceda.ac.